### Dgraph

https://docs.hypermode.com/dgraph/self-managed/single-host-setup#docker-compose


https://github.com/hypermodeinc/pydgraph/blob/main/examples/notebook/self_managed_cluster.ipynb



Conexão com o Spark: https://github.com/G-Research/spark-dgraph-connector

Limitations: 
 * Read-only: The connector does not support mutating the graph.
 * Namespaces: The connector can only read the default namespace
 * Authorization: No authorization against Dgraph supported


Multi-tenancy is an enterprise feature and needs Access Control Lists (ACL) enabled to work.


A Dgraph cluster consists of the following:
 * Dgraph Alpha database server nodes: The Dgraph Alpha server nodes in your deployment host and serve data. These nodes also host an /admin HTTP and gRPC endpoint that can be used for data and node administration tasks such as backup, export, draining, and shutdown.
 * Dgraph Zero management server nodes: The Dgraph Zero nodes in your deployment control the nodes in your Dgraph cluster. Dgraph Zero automatically moves data between different Dgraph Alpha instances based on the volume of data served by each Alpha instance.

In [1]:
!pip install pydgraph    # DQL

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 52.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.10.0
    Uninstalling typing_extensions-4.10.0:
      Successfully uninstalled typing_extensions-4.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.11.2 requires protobuf<4,>=3.9.2, but you have protobuf 6.32.1 which is incompatible.
tensorflow-cpu 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 6.32.1 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
! pip install python-graphql-client   # GraphQL

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pydgraph
import json

# Inicialize o stub do cliente apontando para o endereço do Alpha (padrão: localhost:9080)
client_stub = pydgraph.DgraphClientStub('localhost:9080')

# Crie o cliente
client = pydgraph.DgraphClient(client_stub)


In [5]:
schema = """
name: string @index(exact) .
friend: [uid] .
age: int .
"""
operation = pydgraph.Operation(schema=schema)
client.alter(operation)

In [6]:
# Estrutura dos dados
pessoa = {
    'uid': '_:alice',
    'name': 'Alice',
    'age': 26,
    'friend': [
        {
            'uid': '_:bob',
            'name': 'Bob',
            'age': 24,
        }
    ],
}

# Insira no banco como mutação
txn = client.txn()
try:
    assigned = txn.mutate(set_obj=pessoa)
    txn.commit()
finally:
    txn.discard()

In [18]:
query = """{
  all(func: has(name)) {
    uid
    name
  }
}"""

res = client.txn(read_only=True).query(query)
data = res.json
print(data)


b'{"all":[{"uid":"0x1","name":"Alice"},{"uid":"0x2","name":"Bob"}]}'


In [13]:
# Consultar nós cujo atributo 'name' seja exatamente 'bob'


query = """
{
  nodes(func: eq(name, "Bob")) {
    uid
    name
    age
  }
}
"""

res = client.txn(read_only=True).query(query)
res.json

b'{"nodes":[{"uid":"0x2","name":"Bob","age":24}]}'